# インポート

In [2]:
#インポート文
import numpy as np
import pandas as pd

import pydata_google_auth
from google.cloud import bigquery 

# BigQueryからデータの読み込み

In [10]:
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)
client = bigquery.Client(project='keiba-381006', credentials=credentials)

In [16]:
race_query = '''
    SELECT * 
    FROM satsuki.race_table
    '''
race_df = client.query(race_query).to_dataframe()
race_df.head(1)

,race_id,race_round,race_title,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,race_rank,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,201105020211,11,第71回皐月賞(G1),1,1,2011-04-24,5,18,6,12,2,4,1,2,1080,300,130,550,570,1260,630,3840,1330,3740,9220,55450,3,0,2,1,2000,2011-04-24 15:40:00+00:00


In [15]:
horse_query = '''
    SELECT * 
    FROM satsuki.horse_table 
    '''
horse_df = client.query(horse_query).to_dataframe()
pd.set_option('display.max_columns', 100)
horse_df.head(1)

,race_id,rank,frame_number,horse_number,horse_id,burden_weight,rider_id,goal_time,goal_time_dif,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,is_down,age,sex,horse_weight_dif,burden_weight_rate,avg_velocity
0,202206030811,7,3,6,2019103662,57.0,5386,120.3,0.2,15.5,34,350,10,494,1176,195031,0,3,0,-4,0.115385,16.625104


In [13]:
horse_info_query = '''
    SELECT * 
    FROM satsuki.horse_info_table
    '''
horse_info_df = client.query(horse_info_query).to_dataframe()
pd.set_option('display.max_columns', 100)
horse_info_df.head(1)

,horse_id,bday,tamer_id,owner_id,producer_id,production_area,lifetime_record,wined_race_title,inbreeding_1,inbreeding_2,father,faths_father,faths_mother,mother,moths_father,moths_mother
0,2003110015,2003-03-10,00427,582008,60881.0,米,31戦4勝 [4-2-3-22],200606040211,2005110047,None,000a0000d3,000a000e04,000a000491,000a0101b5,000a001607,000a0085c1


In [14]:
horse_race_query = '''
    SELECT * 
    FROM satsuki.horse_race_table
    '''
horse_race_df = client.query(horse_race_query).to_dataframe()
pd.set_option('display.max_columns', 100)
horse_race_df.head(1)

,date,where_racecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,popular,rank,rider_id,burden_weight,ground_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner_up_horse_id,horse_id,target_race_id,race_rank,horse_weight_dif,is_down,burden_weight_rate,is_obstacle,ground_type,is_left_right_straight,distance,race_date
0,2005-10-30,5,1,7,くるみ賞(500万下),200505040807,9,4,4,25,1,1,700,55.0,1,83.0,0.0,5.5,35.2,34,460.0,2003110138,2003101859,200606030811,0,6.0,0,0.119565,0,2,0,1400,2006-04-16


# EDA

## race_dfの探索

In [17]:
pd.set_option('display.max_columns', 100)
race_df.head(1)

,race_id,race_round,race_title,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,race_rank,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,201105020211,11,第71回皐月賞(G1),1,1,2011-04-24,5,18,6,12,2,4,1,2,1080,300,130,550,570,1260,630,3840,1330,3740,9220,55450,3,0,2,1,2000,2011-04-24 15:40:00+00:00


In [18]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   race_id                 17 non-null     Int64              
 1   race_round              17 non-null     Int64              
 2   race_title              17 non-null     object             
 3   weather                 17 non-null     Int64              
 4   ground_status           17 non-null     Int64              
 5   date                    17 non-null     dbdate             
 6   where_racecourse        17 non-null     Int64              
 7   total_horse_number      17 non-null     Int64              
 8   frame_number_first      17 non-null     Int64              
 9   horse_number_first      17 non-null     Int64              
 10  frame_number_second     17 non-null     Int64              
 11  horse_number_second     17 non-null     Int64  

race_dfには32個の特徴量が存在し、過去の対象レース情報が入っている
それぞれのレースの特徴量から、レースの展開や荒れ具合などの傾向を調べると、競走馬の順位予測の助けになりそうである

### 特徴

#### race_id

In [35]:
race_df['race_id'].unique()

<IntegerArray>
[201105020211, 202006030811, 201406030811, 201506030811, 200606030811,
 200806030811, 201306030811, 201806030811, 202106030811, 201706030811,
 201906030811, 201006030811, 201206030811, 202206030811, 200906030811,
 200706030811, 201606030811]
Length: 17, dtype: Int64

過去17回のレースのデータがInt64型で入っている  
LightGBM Rankerでgroup(検索クエリ)を作成するときに使用する

#### race_round

In [36]:
race_df['race_round'].unique()

<IntegerArray>
[11]
Length: 1, dtype: Int64

11レース目のみなので、削除する

In [37]:
race_df.drop('race_round', axis=1)

,race_id,race_title,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,race_rank,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,201105020211,第71回皐月賞(G1),1,1,2011-04-24,5,18,6,12,2,4,1,2,1080,300,130,550,570,1260,630,3840,1330,3740,9220,55450,3,0,2,1,2000,2011-04-24 15:40:00+00:00
1,202006030811,第80回皐月賞(G1),1,2,2020-04-19,6,18,1,1,4,7,8,16,270,140,170,690,640,660,330,2250,2610,1120,9150,26310,3,0,2,2,2000,2020-04-19 15:40:00+00:00
2,201406030811,第74回皐月賞(G1),2,1,2014-04-20,6,18,1,2,8,17,8,18,510,190,150,470,330,920,390,1990,1250,1900,7690,33490,3,0,2,2,2000,2014-04-20 15:40:00+00:00
3,201506030811,第75回皐月賞(G1),2,1,2015-04-19,6,15,2,2,3,5,4,7,460,190,150,290,780,790,390,1130,860,1530,3110,12360,3,0,2,2,2000,2015-04-19 15:40:00+00:00
4,200606030811,第66回皐月賞(G1),2,1,2006-04-16,6,18,3,5,1,2,3,6,1450,430,760,260,1000,13980,3290,1380,3000,29670,34700,273310,3,0,2,2,2000,2006-04-16 15:40:00+00:00
5,200806030811,第68回皐月賞(G1),2,1,2008-04-20,6,18,3,6,1,1,5,9,1710,430,380,150,2090,10260,2590,1050,940,20300,11230,82750,3,0,2,2,2000,2008-04-20 15:40:00+00:00
6,201306030811,第73回皐月賞(G1),1,1,2013-04-14,6,18,4,7,7,14,6,12,370,140,150,140,440,840,360,320,330,1750,1210,5920,3,0,2,2,2000,2013-04-14 15:40:00+00:00
7,201806030811,第78回皐月賞(G1),2,2,2018-04-15,6,16,4,7,7,14,5,10,1450,430,580,530,9520,12880,3250,3010,4290,23570,53410,372080,3,0,2,2,2000,2018-04-15 15:40:00+00:00
8,202106030811,第81回皐月賞(G1),1,2,2021-04-18,6,16,4,7,7,13,2,3,370,170,490,380,1150,4300,1540,1150,3560,5510,20000,82320,3,0,2,2,2000,2021-04-18 15:40:00+00:00
9,201706030811,第77回皐月賞(G1),1,1,2017-04-16,6,18,6,11,4,7,5,10,2240,650,320,1340,2380,8710,2250,16960,6770,20720,176030,1064360,3,0,2,2,2000,2017-04-16 15:40:00+00:00


#### race_title

In [38]:
race_df['race_title'].unique()

array(['第71回皐月賞(G1)', '第80回皐月賞(G1)', '第74回皐月賞(G1)', '第75回皐月賞(G1)',
       '第66回皐月賞(G1)', '第68回皐月賞(G1)', '第73回皐月賞(G1)', '第78回皐月賞(G1)',
       '第81回皐月賞(G1)', '第77回皐月賞(G1)', '第79回皐月賞(G1)', '第70回皐月賞(G1)',
       '第72回皐月賞(G1)', '第82回皐月賞(G1)', '第69回皐月賞(G1)', '第67回皐月賞(G1)',
       '第76回皐月賞(G1)'], dtype=object)

情報はなさそうなので、削除する

In [39]:
race_df.drop('race_title', axis=1)

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,race_rank,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,201105020211,11,1,1,2011-04-24,5,18,6,12,2,4,1,2,1080,300,130,550,570,1260,630,3840,1330,3740,9220,55450,3,0,2,1,2000,2011-04-24 15:40:00+00:00
1,202006030811,11,1,2,2020-04-19,6,18,1,1,4,7,8,16,270,140,170,690,640,660,330,2250,2610,1120,9150,26310,3,0,2,2,2000,2020-04-19 15:40:00+00:00
2,201406030811,11,2,1,2014-04-20,6,18,1,2,8,17,8,18,510,190,150,470,330,920,390,1990,1250,1900,7690,33490,3,0,2,2,2000,2014-04-20 15:40:00+00:00
3,201506030811,11,2,1,2015-04-19,6,15,2,2,3,5,4,7,460,190,150,290,780,790,390,1130,860,1530,3110,12360,3,0,2,2,2000,2015-04-19 15:40:00+00:00
4,200606030811,11,2,1,2006-04-16,6,18,3,5,1,2,3,6,1450,430,760,260,1000,13980,3290,1380,3000,29670,34700,273310,3,0,2,2,2000,2006-04-16 15:40:00+00:00
5,200806030811,11,2,1,2008-04-20,6,18,3,6,1,1,5,9,1710,430,380,150,2090,10260,2590,1050,940,20300,11230,82750,3,0,2,2,2000,2008-04-20 15:40:00+00:00
6,201306030811,11,1,1,2013-04-14,6,18,4,7,7,14,6,12,370,140,150,140,440,840,360,320,330,1750,1210,5920,3,0,2,2,2000,2013-04-14 15:40:00+00:00
7,201806030811,11,2,2,2018-04-15,6,16,4,7,7,14,5,10,1450,430,580,530,9520,12880,3250,3010,4290,23570,53410,372080,3,0,2,2,2000,2018-04-15 15:40:00+00:00
8,202106030811,11,1,2,2021-04-18,6,16,4,7,7,13,2,3,370,170,490,380,1150,4300,1540,1150,3560,5510,20000,82320,3,0,2,2,2000,2021-04-18 15:40:00+00:00
9,201706030811,11,1,1,2017-04-16,6,18,6,11,4,7,5,10,2240,650,320,1340,2380,8710,2250,16960,6770,20720,176030,1064360,3,0,2,2,2000,2017-04-16 15:40:00+00:00


#### weather

In [40]:
race_df['weather'].unique()

<IntegerArray>
[1, 2]
Length: 2, dtype: Int64

晴は1、曇は2である

#### ground_status

In [41]:
race_df['ground_status'].unique()

<IntegerArray>
[1, 2]
Length: 2, dtype: Int64

良が1、稍と稍重が2である

#### date

In [42]:
race_df['date'].unique()

<DateArray>
[datetime.date(2011, 4, 24), datetime.date(2020, 4, 19),
 datetime.date(2014, 4, 20), datetime.date(2015, 4, 19),
 datetime.date(2006, 4, 16), datetime.date(2008, 4, 20),
 datetime.date(2013, 4, 14), datetime.date(2018, 4, 15),
 datetime.date(2021, 4, 18), datetime.date(2017, 4, 16),
 datetime.date(2019, 4, 14), datetime.date(2010, 4, 18),
 datetime.date(2012, 4, 15), datetime.date(2022, 4, 17),
 datetime.date(2009, 4, 19), datetime.date(2007, 4, 15),
 datetime.date(2016, 4, 17)]
Length: 17, dtype: dbdate

datetime型で、17個のレースの日付が入っている

#### where_racecourse

In [43]:
race_df['where_racecourse'].unique()


<IntegerArray>
[5, 6]
Length: 2, dtype: Int64

第71回皐月賞は東京で開催され、それ以外は中山で開催された

##### total_horse_number

In [44]:
race_df['total_horse_number'].unique()

<IntegerArray>
[18, 15, 16]
Length: 3, dtype: Int64

過去の同レースでは15頭立て以上である

##### frame_number_first, frame_number_second, frame_number_third

In [57]:
print("frame_number_first:" + str(race_df['frame_number_first'].unique()))
print("frame_number_second:" + str(race_df['frame_number_second'].unique()))
print("frame_number_third:" + str(race_df['frame_number_third'].unique()))

frame_number_first:<IntegerArray>
[6, 1, 2, 3, 4, 7, 8]
Length: 7, dtype: Int64
frame_number_second:<IntegerArray>
[2, 4, 8, 3, 1, 7, 5]
Length: 7, dtype: Int64
frame_number_third:<IntegerArray>
[1, 8, 4, 3, 5, 6, 2, 7]
Length: 8, dtype: Int64


##### horse_number_first, horse_number_second, horse_number_third

In [58]:
print("horse_number_first:" + str(race_df['horse_number_first'].unique()))
print("horse_number_second:" + str(race_df['horse_number_second'].unique()))
print("horse_number_third:" + str(race_df['horse_number_third'].unique()))

horse_number_first:<IntegerArray>
[12, 1, 2, 5, 6, 7, 11, 13, 14, 16, 17, 18]
Length: 12, dtype: Int64
horse_number_second:<IntegerArray>
[4, 7, 17, 5, 2, 1, 14, 13, 16, 9, 18, 3]
Length: 12, dtype: Int64
horse_number_third:<IntegerArray>
[2, 16, 18, 7, 6, 9, 12, 10, 3, 4, 11, 15, 1]
Length: 13, dtype: Int64


#### money

今回は割愛
後程、BIで可視化しようと思う

#### race_rank

In [60]:
race_df['race_rank'].unique()

<IntegerArray>
[3]
Length: 1, dtype: Int64

## horse_dfの探索

In [58]:
pd.set_option('display.max_columns', 100)
horse_df.head(1)

,race_id,rank,frame_number,horse_number,horse_id,burden_weight,rider_id,goal_time,goal_time_dif,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,is_down,age,sex,horse_weight_dif,burden_weight_rate,avg_velocity
0,202009020411,2,8,12,2016104750,55,1102,118.4,0.0,3.0,34,5.2,4,454,1151,226800,0,4,1,-6,0.121145,16.891892


In [59]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   race_id             85 non-null     Int64  
 1   rank                85 non-null     Int64  
 2   frame_number        85 non-null     Int64  
 3   horse_number        85 non-null     Int64  
 4   horse_id            85 non-null     Int64  
 5   burden_weight       85 non-null     Int64  
 6   rider_id            85 non-null     Int64  
 7   goal_time           85 non-null     float64
 8   goal_time_dif       85 non-null     float64
 9   half_way_rank       85 non-null     float64
 10  last_time           85 non-null     Int64  
 11  odds                85 non-null     float64
 12  popular             85 non-null     Int64  
 13  horse_weight        85 non-null     Int64  
 14  tamer_id            85 non-null     Int64  
 15  owner_id            85 non-null     Int64  
 16  is_down   

## horse_info_dfの探索

In [60]:
pd.set_option('display.max_columns', 100)
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production_area,auction_price,winnings,lifetime_record,wined_race_title,inbreeding_1,inbreeding_2,father,faths_father,faths_mother,mother,moths_father,moths_mother
0,2012104463,2012年3月17日,01128,471033,393126,千歳市,928万円 (2014年 千葉サラブレッド・セール),"2億3,061万円 (中央)",59戦9勝 [9-8-5-37],201803020811,2011104480,0,2003102205,000a000d77,1997102385,2004103210,1998101516,000a00fa62


In [61]:
horse_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   horse_id          85 non-null     Int64 
 1   bday              85 non-null     object
 2   tame_id           85 non-null     object
 3   owner_id          85 non-null     Int64 
 4   producer_id       52 non-null     Int64 
 5   production_area   85 non-null     object
 6   auction_price     85 non-null     object
 7   winnings          85 non-null     object
 8   lifetime_record   85 non-null     object
 9   wined_race_title  85 non-null     Int64 
 10  inbreeding_1      85 non-null     object
 11  inbreeding_2      85 non-null     Int64 
 12  father            85 non-null     object
 13  faths_father      85 non-null     object
 14  faths_mother      85 non-null     object
 15  mother            85 non-null     object
 16  moths_father      85 non-null     object
 17  moths_mother      

In [62]:
horse_info_df['inbreeding_2']

0              0
1     2008102792
2     2014105768
3     2011103737
4     2013106007
         ...    
80             0
81             0
82             0
83             0
84             0
Name: inbreeding_2, Length: 85, dtype: Int64

In [63]:
horse_info_df['inbreeding_2'].replace(0, np.nan, inplace=True)

In [68]:
horse_info_df['inbreeding_2']
horse_info_df['inbreeding_2'] = horse_info_df['inbreeding_2'].astype(str)
horse_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   horse_id          85 non-null     Int64 
 1   bday              85 non-null     object
 2   tame_id           85 non-null     object
 3   owner_id          85 non-null     Int64 
 4   producer_id       52 non-null     Int64 
 5   production_area   85 non-null     object
 6   auction_price     85 non-null     object
 7   winnings          85 non-null     object
 8   lifetime_record   85 non-null     object
 9   wined_race_title  85 non-null     Int64 
 10  inbreeding_1      85 non-null     object
 11  inbreeding_2      85 non-null     object
 12  father            85 non-null     object
 13  faths_father      85 non-null     object
 14  faths_mother      85 non-null     object
 15  mother            85 non-null     object
 16  moths_father      85 non-null     object
 17  moths_mother      

## horse_race_dfの探索

In [65]:
pd.set_option('display.max_columns', 100)
horse_race_df.head(1)

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,popular,rank,rider_id,horse_wight,distance,groud_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner_up_horse_id,prize,horse_id,race_type,surface,burden_weight,horse_weight_dif
0,2001-04-07,3中山5,晴,11,NZT(G2),200106030511,14,6,10,2.0,1,2,5022,56.0,1600,良,95.8,0.1,9-9-11,36.0-35.2,34.6,468.0,1.998104e+09,"2,232.4",1998107227,G2,芝,56.0,-6.0


In [66]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7374 entries, 0 to 7373
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                7374 non-null   dbdate 
 1   whereracecourse     7374 non-null   object 
 2   weather             7374 non-null   object 
 3   race_round          7374 non-null   object 
 4   race_title          7374 non-null   object 
 5   race_id             7374 non-null   object 
 6   total_horse_number  7374 non-null   Int64  
 7   frame_number        7374 non-null   object 
 8   horse_number        7374 non-null   Int64  
 9   odds                7374 non-null   float64
 10  popular             7374 non-null   Int64  
 11  rank                7374 non-null   Int64  
 12  rider_id            7374 non-null   Int64  
 13  horse_wight         7374 non-null   float64
 14  distance            7374 non-null   Int64  
 15  groud_status        7374 non-null   object 
 16  goal_t